# Merging 

start by importing packages and setting up some settings

In [1]:
#import packages
import pandas as pd
import numpy as np


#see all columns and up to 150 rows as needed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

# Load in the patentsview.org combined dataset

In [2]:
#load in the data
patents = pd.read_csv('reduced_2009_2019.csv')

/Applications/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
patents.patent_number = patents.patent_number.astype('str')
patents.dtypes


Unnamed: 0             int64
patent_number         object
assignee              object
grant_year             int64
application_year       int64
application_number     int64
country               object
city                  object
state                 object
county                object
ipc_section           object
team_size              int64
inventors              int64
men_inventors          int64
women_inventors        int64
dtype: object

In [5]:
#drop the old index column
patents = patents.drop('Unnamed: 0', axis =1 )

In [6]:
#check out that it worked 
patents

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors
0,7472428,"NIKE, INC.",2009,2004,10814594,US,Beaverton,OR,Washington,A,2,2,2,0
1,7472430,WATKINS MANUFACTURING CORPORATION,2009,2006,11604596,US,Vista,CA,San Diego,E,4,4,4,0
2,7472431,WATKINS MANUFACTURING CORPORATION,2009,2004,10899547,US,Vista,CA,San Diego,A,1,1,1,0
3,7472433,MASCO CORPORATION OF INDIANA,2009,2006,11325927,US,Indianapolis,IN,Marion,E F G,2,2,2,0
4,7472434,"T&S BRASS AND BRONZE WORKS, INC.",2009,2008,12052249,US,Travelers Rest,SC,Greenville,E,3,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533890,RE47549,"S.R. SMITH, LLC",2019,2015,29515597,US,Canby,OR,Clackamas,NaN,2,2,2,0
1533891,RE47572,"THE CHARLES MACHINE WORKS, INC.",2019,2018,29655614,US,Perry,OK,Noble,NaN,2,2,2,0
1533892,RE47657,"TOSHIBA AMERICA BUSINESS SOLUTIONS, INC.",2019,2017,15845923,US,Lake Forest,CA,Orange,G,4,4,4,0
1533893,RE47664,APPLE INC.,2019,2017,29622909,US,Cupertino,CA,Santa Clara,NaN,14,14,14,0


In [7]:
patents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1533895 entries, 0 to 1533894
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   patent_number       1533895 non-null  object
 1   assignee            1533895 non-null  object
 2   grant_year          1533895 non-null  int64 
 3   application_year    1533895 non-null  int64 
 4   application_number  1533895 non-null  int64 
 5   country             1533895 non-null  object
 6   city                1532744 non-null  object
 7   state               1533850 non-null  object
 8   county              1433937 non-null  object
 9   ipc_section         1412685 non-null  object
 10  team_size           1533895 non-null  int64 
 11  inventors           1533895 non-null  int64 
 12  men_inventors       1533895 non-null  int64 
 13  women_inventors     1533895 non-null  int64 
dtypes: int64(7), object(7)
memory usage: 163.8+ MB


In [8]:
#about 100,000 don't have counties -> so need to merge on something else
sum(patents.county.isna())

99958

In [10]:
#upper case to besure of any matching issues
patents.city = patents.city.str.strip().str.upper()
#patents

# Load in crosswalks to start to merge

crosswalk only for city,state for places without county -> trying to get the geoid so can get county later 

In [19]:
# city_crosswalk = pd.read_csv('crosswalk_city_state_name_to_city_state_fips.csv')
city_crosswalk = pd.read_csv('crosswalk_city_state_name_to_city_state_fips.csv')

#upper case to be sure match
city_crosswalk.city = city_crosswalk.city.str.strip().str.upper()
city_crosswalk.head()

,state,city,STATEFP,COUNTYFP,GEOID
0,AE,APO,35.0,57.0,35057
1,AK,ALEXANDER,40.0,51.0,00051
2,AK,CENTERTON,5.0,7.0,05007
3,AK,FORT SMITH,5.0,131.0,05131
4,AK,MONTICELLO,27.0,171.0,27171


In [14]:
#test it out
# test_merge = pd.merge(patents, city_crosswalk,
#                       on = ['city', 'state'],
#                       how = 'outer',
#                       indicator = 'matched',
#                       validate = 'many_to_one')

#not all match becasue crosswalk only made for places without a county
#len(test_merge.query("matched == 'left_only'").city.unique())

In [13]:
#pd.DataFrame(test_merge.query("matched == 'left_only'").groupby(['city', 'state']).size()).sort_values(by='state')

## Most of the patents don't get a GEOID -> just for places with no county

In [20]:
#now do it for real 

patents_merged = pd.merge(patents, city_crosswalk,
                          on = ['city', 'state'],
                          how = 'left')

patents_merged
#now everywhere has a city/state  
#patents_merged.query("county.isnull() & GEOID.isnull()", engine = 'python').groupby('state').size().sort_values(ascending = False)

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,STATEFP,COUNTYFP,GEOID
0,7472428,"NIKE, INC.",2009,2004,10814594,US,BEAVERTON,OR,Washington,A,2,2,2,0,NaN,NaN,NaN
1,7472430,WATKINS MANUFACTURING CORPORATION,2009,2006,11604596,US,VISTA,CA,San Diego,E,4,4,4,0,NaN,NaN,NaN
2,7472431,WATKINS MANUFACTURING CORPORATION,2009,2004,10899547,US,VISTA,CA,San Diego,A,1,1,1,0,NaN,NaN,NaN
3,7472433,MASCO CORPORATION OF INDIANA,2009,2006,11325927,US,INDIANAPOLIS,IN,Marion,E F G,2,2,2,0,NaN,NaN,NaN
4,7472434,"T&S BRASS AND BRONZE WORKS, INC.",2009,2008,12052249,US,TRAVELERS REST,SC,Greenville,E,3,3,3,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533890,RE47549,"S.R. SMITH, LLC",2019,2015,29515597,US,CANBY,OR,Clackamas,NaN,2,2,2,0,NaN,NaN,NaN
1533891,RE47572,"THE CHARLES MACHINE WORKS, INC.",2019,2018,29655614,US,PERRY,OK,Noble,NaN,2,2,2,0,NaN,NaN,NaN
1533892,RE47657,"TOSHIBA AMERICA BUSINESS SOLUTIONS, INC.",2019,2017,15845923,US,LAKE FOREST,CA,Orange,G,4,4,4,0,NaN,NaN,NaN
1533893,RE47664,APPLE INC.,2019,2017,29622909,US,CUPERTINO,CA,Santa Clara,NaN,14,14,14,0,NaN,NaN,NaN


In [21]:
#didn't work for some becasue not in US, and/or missing sityc, etc
patents_merged.query("county.isnull() & GEOID.isnull()", engine = 'python')#.groupby('state').size().sort_values(ascending = False)

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,STATEFP,COUNTYFP,GEOID
1458,7475569,"THE BOC GRPOUP, INC.",2009,2003,10712904,US,NaN,NJ,NaN,C,3,3,3,0,NaN,NaN,NaN
8403,7490885,"NISSAN TECHNICAL CENTER NORTH AMERICA, INC.",2009,2008,12037624,US,NaN,MI,NaN,B,1,1,0,1,NaN,NaN,NaN
8439,7491002,"CRITERION MANUFACTURING SOLUTIONS, INC.",2009,2005,11088736,US,NaN,GA,NaN,G,3,3,3,0,NaN,NaN,NaN
9992,7494403,EDGECRAFT CORPORATION,2009,2007,11676597,US,NaN,PA,NaN,B,1,1,1,0,NaN,NaN,NaN
11784,7498279,"OWENS CORNING INTELLECTUAL CAPITAL, LLC",2009,2007,11725344,US,NaN,DE,NaN,B,1,1,1,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518517,10416495,HISENSE USA CORPORATION,2019,2017,15853598,US,NaN,GA,NaN,G,1,1,1,0,NaN,NaN,NaN
1520553,D839809,"TURN 5, INC.",2019,2017,29602013,US,NaN,PA,NaN,NaN,1,1,1,0,NaN,NaN,NaN
1525550,D851425,"KEEZIO GROUP, LLC",2019,2017,29623261,US,NaN,CA,NaN,NaN,1,1,1,0,NaN,NaN,NaN
1530280,D862502,LEARNING CIRCLE KIDS LLC,2019,2017,29591207,US,NaN,WA,NaN,NaN,3,3,0,3,NaN,NaN,NaN


In [23]:
#keep places that have county or GEOID
patents_merged = patents_merged[~(patents_merged["county"].isnull() & patents_merged["GEOID"].isnull())]
patents_merged

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,STATEFP,COUNTYFP,GEOID
0,7472428,"NIKE, INC.",2009,2004,10814594,US,BEAVERTON,OR,Washington,A,2,2,2,0,NaN,NaN,NaN
1,7472430,WATKINS MANUFACTURING CORPORATION,2009,2006,11604596,US,VISTA,CA,San Diego,E,4,4,4,0,NaN,NaN,NaN
2,7472431,WATKINS MANUFACTURING CORPORATION,2009,2004,10899547,US,VISTA,CA,San Diego,A,1,1,1,0,NaN,NaN,NaN
3,7472433,MASCO CORPORATION OF INDIANA,2009,2006,11325927,US,INDIANAPOLIS,IN,Marion,E F G,2,2,2,0,NaN,NaN,NaN
4,7472434,"T&S BRASS AND BRONZE WORKS, INC.",2009,2008,12052249,US,TRAVELERS REST,SC,Greenville,E,3,3,3,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533890,RE47549,"S.R. SMITH, LLC",2019,2015,29515597,US,CANBY,OR,Clackamas,NaN,2,2,2,0,NaN,NaN,NaN
1533891,RE47572,"THE CHARLES MACHINE WORKS, INC.",2019,2018,29655614,US,PERRY,OK,Noble,NaN,2,2,2,0,NaN,NaN,NaN
1533892,RE47657,"TOSHIBA AMERICA BUSINESS SOLUTIONS, INC.",2019,2017,15845923,US,LAKE FOREST,CA,Orange,G,4,4,4,0,NaN,NaN,NaN
1533893,RE47664,APPLE INC.,2019,2017,29622909,US,CUPERTINO,CA,Santa Clara,NaN,14,14,14,0,NaN,NaN,NaN


# now merge with states info

other data only has state abreeviations, also need state name to match with later dataset

In [29]:
# state abbreviation to state name

states_cross = pd.read_csv("stateabbr_statename.csv")
states_cross.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


## territories were included in the data -> only interested in US so got rid of the other places

- examples: Puerto Rico, Guam, etc 

In [30]:
#merge state cross to patents 
test_merge = pd.merge(patents_merged, states_cross,
                      left_on = 'state',
                      right_on = 'Code',
                      how = 'outer',
                      indicator = 'matched',
                      validate = 'many_to_one')

#some stuff that's not a state
test_merge.query("matched != 'both'").groupby('state').size()

state
AE       1
AS       3
GU      55
MP       4
PR    1496
VI      67
dtype: int64

In [31]:
patents_merged = pd.merge(patents_merged, states_cross,
                      left_on = 'state',
                      right_on = 'Code',
                      how = 'left')
patents_merged

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,STATEFP,COUNTYFP,GEOID,State,Abbrev,Code
0,7472428,"NIKE, INC.",2009,2004,10814594,US,BEAVERTON,OR,Washington,A,2,2,2,0,NaN,NaN,NaN,Oregon,Ore.,OR
1,7472430,WATKINS MANUFACTURING CORPORATION,2009,2006,11604596,US,VISTA,CA,San Diego,E,4,4,4,0,NaN,NaN,NaN,California,Calif.,CA
2,7472431,WATKINS MANUFACTURING CORPORATION,2009,2004,10899547,US,VISTA,CA,San Diego,A,1,1,1,0,NaN,NaN,NaN,California,Calif.,CA
3,7472433,MASCO CORPORATION OF INDIANA,2009,2006,11325927,US,INDIANAPOLIS,IN,Marion,E F G,2,2,2,0,NaN,NaN,NaN,Indiana,Ind.,IN
4,7472434,"T&S BRASS AND BRONZE WORKS, INC.",2009,2008,12052249,US,TRAVELERS REST,SC,Greenville,E,3,3,3,0,NaN,NaN,NaN,South Carolina,S.C.,SC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1532727,RE47549,"S.R. SMITH, LLC",2019,2015,29515597,US,CANBY,OR,Clackamas,NaN,2,2,2,0,NaN,NaN,NaN,Oregon,Ore.,OR
1532728,RE47572,"THE CHARLES MACHINE WORKS, INC.",2019,2018,29655614,US,PERRY,OK,Noble,NaN,2,2,2,0,NaN,NaN,NaN,Oklahoma,Okla.,OK
1532729,RE47657,"TOSHIBA AMERICA BUSINESS SOLUTIONS, INC.",2019,2017,15845923,US,LAKE FOREST,CA,Orange,G,4,4,4,0,NaN,NaN,NaN,California,Calif.,CA
1532730,RE47664,APPLE INC.,2019,2017,29622909,US,CUPERTINO,CA,Santa Clara,NaN,14,14,14,0,NaN,NaN,NaN,California,Calif.,CA


In [33]:
patents_merged.State = patents_merged.State.str.strip().str.upper()
patents_merged.county = patents_merged.county.str.strip().str.upper()
patents_merged

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,STATEFP,COUNTYFP,GEOID,State,Abbrev,Code
0,7472428,"NIKE, INC.",2009,2004,10814594,US,BEAVERTON,OR,WASHINGTON,A,2,2,2,0,NaN,NaN,NaN,OREGON,Ore.,OR
1,7472430,WATKINS MANUFACTURING CORPORATION,2009,2006,11604596,US,VISTA,CA,SAN DIEGO,E,4,4,4,0,NaN,NaN,NaN,CALIFORNIA,Calif.,CA
2,7472431,WATKINS MANUFACTURING CORPORATION,2009,2004,10899547,US,VISTA,CA,SAN DIEGO,A,1,1,1,0,NaN,NaN,NaN,CALIFORNIA,Calif.,CA
3,7472433,MASCO CORPORATION OF INDIANA,2009,2006,11325927,US,INDIANAPOLIS,IN,MARION,E F G,2,2,2,0,NaN,NaN,NaN,INDIANA,Ind.,IN
4,7472434,"T&S BRASS AND BRONZE WORKS, INC.",2009,2008,12052249,US,TRAVELERS REST,SC,GREENVILLE,E,3,3,3,0,NaN,NaN,NaN,SOUTH CAROLINA,S.C.,SC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1532727,RE47549,"S.R. SMITH, LLC",2019,2015,29515597,US,CANBY,OR,CLACKAMAS,NaN,2,2,2,0,NaN,NaN,NaN,OREGON,Ore.,OR
1532728,RE47572,"THE CHARLES MACHINE WORKS, INC.",2019,2018,29655614,US,PERRY,OK,NOBLE,NaN,2,2,2,0,NaN,NaN,NaN,OKLAHOMA,Okla.,OK
1532729,RE47657,"TOSHIBA AMERICA BUSINESS SOLUTIONS, INC.",2019,2017,15845923,US,LAKE FOREST,CA,ORANGE,G,4,4,4,0,NaN,NaN,NaN,CALIFORNIA,Calif.,CA
1532730,RE47664,APPLE INC.,2019,2017,29622909,US,CUPERTINO,CA,SANTA CLARA,NaN,14,14,14,0,NaN,NaN,NaN,CALIFORNIA,Calif.,CA


# Need to pull out placed that don't have a county listed

- They can NOT join on county/ state like others
- will need to join on GEOID separately 

In [26]:
#pull out all places with no county 

patents_no_county = patents_merged.query("county.isnull()", engine = 'python')
#merge to crosswalk separtely using GeoID
patents_no_county

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,STATEFP,COUNTYFP,GEOID
46,7472531,"FLEECEWORKS, INC.",2009,2005,11218805,US,CITY OF INDUSTRY,CA,NaN,B,1,1,0,1,6.0,37.0,06037
59,7472557,CARRIER CORPORATION,2009,2004,11025351,US,FARMINGTON,CT,NaN,F,4,3,3,0,35.0,45.0,35045
93,7472657,"FUEL AND FURNACE CONSULTING, INC.",2009,2006,11383245,US,VENETIA,PA,NaN,F,2,2,2,0,1.0,97.0,01097
102,7472687,"VISTEON GLOBAL TECHNOLOGIES, INC.",2009,2007,11683735,US,VAN BUREN TOWNSHIP,MI,NaN,F,2,1,1,0,26.0,163.0,26163
113,7472716,FISHER CONTROLS INTERNATIONAL LLC,2009,2007,11671808,US,ST. LOUIS,MO,NaN,F,1,1,1,0,29.0,510.0,29005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533770,D871374,"HARMAN INTERNATIONAL INDUSTRIES, INCORPORATED",2019,2017,29631217,US,NORTHRIDGE,CA,NaN,NaN,2,2,1,1,39.0,113.0,39113
1533771,D871375,"HARMAN INTERNATIONAL INDUSTRIES, INCORPORATED",2019,2017,29631258,US,NORTHRIDGE,CA,NaN,NaN,1,1,0,1,39.0,113.0,39113
1533829,D871544,EMERSON ELECTRIC CO.,2019,2019,29691933,US,ST. LOUIS,MO,NaN,NaN,2,2,2,0,29.0,510.0,29005
1533854,D871616,"LUMINAIRE LED, INC.",2019,2017,29608402,US,EDISON,NJ,NaN,NaN,2,2,2,0,34.0,23.0,34023


## Get the county cross walk ready

In [27]:
#load county crosswalk
county_cross = pd.read_csv('crosswalk_county_name_to_county_fips.csv')
county_cross.state_name = county_cross.state_name.str.strip().str.upper()
county_cross.county_name = county_cross.county_name.str.strip().str.upper()
county_cross

,STATEFP,COUNTYFP,GEOID,county_name,state_name
0,31,39,31039,CUMING,NEBRASKA
1,53,69,53069,WAHKIAKUM,WASHINGTON
2,35,11,35011,DE BACA,NEW MEXICO
3,31,109,31109,LANCASTER,NEBRASKA
4,31,129,31129,NUCKOLLS,NEBRASKA
...,...,...,...,...,...
3229,13,123,13123,GILMER,GEORGIA
3230,27,135,27135,ROSEAU,MINNESOTA
3231,28,89,28089,MADISON,MISSISSIPPI
3232,48,227,48227,HOWARD,TEXAS


In [34]:
#county_cross.query("county_name == 'ST. LOUIS'") #& state_name == 'VIRGINIA'")

## Duplicates in crosswalk

some cities are independent of the county -> dropping the city geoid's
drop:
- BALTIMORE, Marylan with GEOID 24510
- FRANKLIN, VIRGINIA 51620
- 51770 ROANOKE, VIRGINIA
- 51600 FAIRFAX, VIRGINIA
- 51760 RICHMOND, VIRGINIA
- 29510 ST. LOUIS, MISSOUR

In [35]:
drop_list = [24510, 51620, 51770, 51600, 51760 ,29510] 


In [36]:
county_cross = county_cross.query("GEOID != @drop_list")
county_cross

,STATEFP,COUNTYFP,GEOID,county_name,state_name
0,31,39,31039,CUMING,NEBRASKA
1,53,69,53069,WAHKIAKUM,WASHINGTON
2,35,11,35011,DE BACA,NEW MEXICO
3,31,109,31109,LANCASTER,NEBRASKA
4,31,129,31129,NUCKOLLS,NEBRASKA
...,...,...,...,...,...
3229,13,123,13123,GILMER,GEORGIA
3230,27,135,27135,ROSEAU,MINNESOTA
3231,28,89,28089,MADISON,MISSISSIPPI
3232,48,227,48227,HOWARD,TEXAS


In [43]:
#make sure GEOID is a string so don't lose leading zeros 
county_cross = county_cross.assign(GEOID = county_cross.GEOID.astype('str'))

#check that it worked 
county_cross.dtypes

#save without the duplicates for independent cities vs counties
#county_cross.to_csv("crosswalk_county_name_to_county_fips.csv")

STATEFP         int64
COUNTYFP        int64
GEOID          object
county_name    object
state_name     object
dtype: object

In [45]:
# practice merge for the nocounty on GEOID
# test_merge = pd.merge(patents_no_county, county_cross,
#                       how = 'outer',
#                       on = 'GEOID',
#                       indicator = 'matched',
#                       validate = 'many_to_one')

# pd.DataFrame(test_merge.query("matched == 'left_only'").groupby(['city','state']).size()).sort_values(by = 0, ascending = False).head(30)

In [46]:
#merge no county patents with county codes on GEOID
patents_no_county_merge = pd.merge(patents_no_county, county_cross,
                                   how = 'inner',
                                   on = 'GEOID')
patents_no_county_merge

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,STATEFP_x,COUNTYFP_x,GEOID,STATEFP_y,COUNTYFP_y,county_name,state_name
0,7472557,CARRIER CORPORATION,2009,2004,11025351,US,FARMINGTON,CT,NaN,F,4,3,3,0,35.0,45.0,35045,35,45,SAN JUAN,NEW MEXICO
1,7472744,CARRIER CORPORATION,2009,2005,11794273,US,FARMINGTON,CT,NaN,F,2,1,1,0,35.0,45.0,35045,35,45,SAN JUAN,NEW MEXICO
2,7475757,OTIS ELEVATOR COMPANY,2009,2003,10554131,US,FARMINGTON,CT,NaN,B,2,2,1,1,35.0,45.0,35045,35,45,SAN JUAN,NEW MEXICO
3,7479861,OTIS ELEVATOR COMPANY,2009,2003,10552383,US,FARMINGTON,CT,NaN,H,10,10,8,2,35.0,45.0,35045,35,45,SAN JUAN,NEW MEXICO
4,7481069,CARRIER CORPORATION,2009,2005,11191679,US,FARMINGTON,CT,NaN,F,2,2,2,0,35.0,45.0,35045,35,45,SAN JUAN,NEW MEXICO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76460,10500717,"STILWELL TECHNOLOGY AND ROBOTICS, LLC",2019,2017,15599248,US,BRANDY STATION,VA,NaN,B G,3,3,3,0,51.0,47.0,51047,51,47,CULPEPER,VIRGINIA
76461,10501486,DOW TECHNOLOGY INVESTMENTS LLC,2019,2015,15310031,US,MIDLANT,MI,NaN,B C,4,4,4,0,48.0,329.0,48329,48,329,MIDLAND,TEXAS
76462,10517282,"JAMES E. BAKER, JR.",2019,2019,16262202,US,"RUSSELVILLE, MONITEAU COUNTY",MO,NaN,A,1,1,1,0,29.0,135.0,29135,29,135,MONITEAU,MISSOURI
76463,D844044,"DANGO DESIGN, LLC",2019,2017,29592731,US,COCALALLA,ID,NaN,NaN,1,1,1,0,16.0,17.0,16017,16,17,BONNER,IDAHO


## working through the merge

- commented out stuff is from before I removed the patents with no county info

In [48]:
# test_merge = pd.merge(patents_merged, county_cross,
#                       how = 'outer',
#                       left_on = ['county', 'State'],
#                       right_on = ['county_name', 'state_name'],
#                       indicator = 'matched',
#                       validate = 'many_to_one')


In [49]:
#test_merge.query("matched == 'left_only'")

In [50]:
# play = pd.DataFrame(test_merge.query("matched == 'left_only'").groupby(['city', 'state']).size())
# #.query("0 > 1000")#.sort_values(by = 0, ascending = False)
# play = play.rename({0: 'number'}, axis = 1)
# play

In [51]:
#play.query("number > 100").sort_values(by= 'number', ascending = False)

## this is the merge for patents WITH counties on county and state


In [52]:
patents_merged = pd.merge(patents_merged, county_cross,
                      how = 'inner',
                      left_on = ['county', 'State'],
                      right_on = ['county_name', 'state_name'])
patents_merged

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,STATEFP_x,COUNTYFP_x,GEOID_x,State,Abbrev,Code,STATEFP_y,COUNTYFP_y,GEOID_y,county_name,state_name
0,7472428,"NIKE, INC.",2009,2004,10814594,US,BEAVERTON,OR,WASHINGTON,A,2,2,2,0,NaN,NaN,NaN,OREGON,Ore.,OR,41,67,41067,WASHINGTON,OREGON
1,7472496,"NIKE, INC.",2009,2005,11055158,US,BEAVERTON,OR,WASHINGTON,A,2,2,1,1,NaN,NaN,NaN,OREGON,Ore.,OR,41,67,41067,WASHINGTON,OREGON
2,7472634,"SD3, LLC",2009,2004,10923290,US,TUALATIN,OR,WASHINGTON,B,3,3,3,0,NaN,NaN,NaN,OREGON,Ore.,OR,41,67,41067,WASHINGTON,OREGON
3,7474419,FEI COMPANY,2009,2006,11590583,US,HILLSBORO,OR,WASHINGTON,G,6,6,5,1,NaN,NaN,NaN,OREGON,Ore.,OR,41,67,41067,WASHINGTON,OREGON
4,7474972,"TEKTRONIX, INC.",2009,2007,11690797,US,BEAVERTON,OR,WASHINGTON,G,3,3,3,0,NaN,NaN,NaN,OREGON,Ore.,OR,41,67,41067,WASHINGTON,OREGON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422221,D858729,"PELLET FIRE PIT, LLC",2019,2017,29592412,US,WEBSTER CITY,IA,HAMILTON,NaN,1,1,1,0,NaN,NaN,NaN,IOWA,Iowa,IA,19,79,19079,HAMILTON,IOWA
1422222,D858794,"J & J PORTABLE SANITATION PRODUCTS, LLC",2019,2018,29639264,US,CRAWFORD,GA,OGLETHORPE,NaN,4,4,3,1,NaN,NaN,NaN,GEORGIA,Ga.,GA,13,221,13221,OGLETHORPE,GEORGIA
1422223,D861112,2 LOTT'S L.L.C.,2019,2017,29598211,US,MINNEAPOLIS,KS,OTTAWA,NaN,2,2,2,0,NaN,NaN,NaN,KANSAS,Kans.,KS,20,143,20143,OTTAWA,KANSAS
1422224,D868611,WALKABOUT DEVELOPMENTS LLC,2019,2018,29656834,US,FAULKTON,SD,FAULK,NaN,1,1,1,0,NaN,NaN,NaN,SOUTH DAKOTA,S.D.,SD,46,49,46049,FAULK,SOUTH DAKOTA


In [54]:
# working here 
#common = pd.merge(patents_no_county_merge, patents_merged,
#                  on=["patent_number"])
#common
#result = dataframe1[~dataframe1.column1.isin(common.column1)]

In [58]:
patents_merged = pd.concat([patents_no_county_merge, patents_merged], ignore_index=True)
#sum(practice.duplicated())

In [59]:
patents_merged.columns

Index(['patent_number', 'assignee', 'grant_year', 'application_year',
       'application_number', 'country', 'city', 'state', 'county',
       'ipc_section', 'team_size', 'inventors', 'men_inventors',
       'women_inventors', 'STATEFP_x', 'COUNTYFP_x', 'GEOID', 'STATEFP_y',
       'COUNTYFP_y', 'county_name', 'state_name', 'GEOID_x', 'State', 'Abbrev',
       'Code', 'GEOID_y'],
      dtype='object')

In [61]:
patents_merged= patents_merged[['patent_number', 'assignee', 'grant_year', 'application_year',
       'application_number', 'country', 'city', 'state', 'county',
       'ipc_section', 'team_size', 'inventors', 'men_inventors',
       'women_inventors','GEOID_y', 'county_name','state_name', 'GEOID']]

patents_merged

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,GEOID_y,county_name,state_name,GEOID
0,7472557,CARRIER CORPORATION,2009,2004,11025351,US,FARMINGTON,CT,NaN,F,4,3,3,0,NaN,SAN JUAN,NEW MEXICO,35045
1,7472744,CARRIER CORPORATION,2009,2005,11794273,US,FARMINGTON,CT,NaN,F,2,1,1,0,NaN,SAN JUAN,NEW MEXICO,35045
2,7475757,OTIS ELEVATOR COMPANY,2009,2003,10554131,US,FARMINGTON,CT,NaN,B,2,2,1,1,NaN,SAN JUAN,NEW MEXICO,35045
3,7479861,OTIS ELEVATOR COMPANY,2009,2003,10552383,US,FARMINGTON,CT,NaN,H,10,10,8,2,NaN,SAN JUAN,NEW MEXICO,35045
4,7481069,CARRIER CORPORATION,2009,2005,11191679,US,FARMINGTON,CT,NaN,F,2,2,2,0,NaN,SAN JUAN,NEW MEXICO,35045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498686,D858729,"PELLET FIRE PIT, LLC",2019,2017,29592412,US,WEBSTER CITY,IA,HAMILTON,NaN,1,1,1,0,19079,HAMILTON,IOWA,NaN
1498687,D858794,"J & J PORTABLE SANITATION PRODUCTS, LLC",2019,2018,29639264,US,CRAWFORD,GA,OGLETHORPE,NaN,4,4,3,1,13221,OGLETHORPE,GEORGIA,NaN
1498688,D861112,2 LOTT'S L.L.C.,2019,2017,29598211,US,MINNEAPOLIS,KS,OTTAWA,NaN,2,2,2,0,20143,OTTAWA,KANSAS,NaN
1498689,D868611,WALKABOUT DEVELOPMENTS LLC,2019,2018,29656834,US,FAULKTON,SD,FAULK,NaN,1,1,1,0,46049,FAULK,SOUTH DAKOTA,NaN


# get geoids into one column 

In [63]:
patents_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498691 entries, 0 to 1498690
Data columns (total 18 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   patent_number       1498691 non-null  object
 1   assignee            1498691 non-null  object
 2   grant_year          1498691 non-null  int64 
 3   application_year    1498691 non-null  int64 
 4   application_number  1498691 non-null  int64 
 5   country             1498691 non-null  object
 6   city                1498691 non-null  object
 7   state               1498691 non-null  object
 8   county              1422226 non-null  object
 9   ipc_section         1381395 non-null  object
 10  team_size           1498691 non-null  int64 
 11  inventors           1498691 non-null  int64 
 12  men_inventors       1498691 non-null  int64 
 13  women_inventors     1498691 non-null  int64 
 14  GEOID_y             1422226 non-null  object
 15  county_name         1498691 non-

In [70]:
patents_merged = patents_merged.assign(GEOID_y = patents_merged.GEOID_y.astype('str'))

In [71]:
type(patents_merged.GEOID_y[0])

str

In [83]:
patents_merged.GEOID_y = patents_merged.GEOID_y.replace('nan', np.nan)
patents_merged.GEOID_y[0]

nan

In [84]:
pd.isnull(patents_merged.GEOID_y[0])

True

In [89]:
pd.isnull(patents_merged.loc[1,'GEOID_y'])

False

In [87]:
for i in range(len(patents_merged)):
    if pd.isnull(patents_merged.loc[i,'GEOID_y']):
        patents_merged.loc[i, 'GEOID_y'] =  patents_merged.loc[i, 'GEOID']
    else:
        pass
    
patents_merged

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,GEOID_y,county_name,state_name,GEOID
0,7472557,CARRIER CORPORATION,2009,2004,11025351,US,FARMINGTON,CT,NaN,F,4,3,3,0,35045,SAN JUAN,NEW MEXICO,35045
1,7472744,CARRIER CORPORATION,2009,2005,11794273,US,FARMINGTON,CT,NaN,F,2,1,1,0,35045,SAN JUAN,NEW MEXICO,35045
2,7475757,OTIS ELEVATOR COMPANY,2009,2003,10554131,US,FARMINGTON,CT,NaN,B,2,2,1,1,35045,SAN JUAN,NEW MEXICO,35045
3,7479861,OTIS ELEVATOR COMPANY,2009,2003,10552383,US,FARMINGTON,CT,NaN,H,10,10,8,2,35045,SAN JUAN,NEW MEXICO,35045
4,7481069,CARRIER CORPORATION,2009,2005,11191679,US,FARMINGTON,CT,NaN,F,2,2,2,0,35045,SAN JUAN,NEW MEXICO,35045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498686,D858729,"PELLET FIRE PIT, LLC",2019,2017,29592412,US,WEBSTER CITY,IA,HAMILTON,NaN,1,1,1,0,19079,HAMILTON,IOWA,NaN
1498687,D858794,"J & J PORTABLE SANITATION PRODUCTS, LLC",2019,2018,29639264,US,CRAWFORD,GA,OGLETHORPE,NaN,4,4,3,1,13221,OGLETHORPE,GEORGIA,NaN
1498688,D861112,2 LOTT'S L.L.C.,2019,2017,29598211,US,MINNEAPOLIS,KS,OTTAWA,NaN,2,2,2,0,20143,OTTAWA,KANSAS,NaN
1498689,D868611,WALKABOUT DEVELOPMENTS LLC,2019,2018,29656834,US,FAULKTON,SD,FAULK,NaN,1,1,1,0,46049,FAULK,SOUTH DAKOTA,NaN


In [92]:
patents_merged = patents_merged.rename({'GEOID_y' :'geoID'}, axis = 1)

In [93]:
patents_merged.columns

Index(['patent_number', 'assignee', 'grant_year', 'application_year',
       'application_number', 'country', 'city', 'state', 'county',
       'ipc_section', 'team_size', 'inventors', 'men_inventors',
       'women_inventors', 'geoID', 'county_name', 'state_name', 'GEOID'],
      dtype='object')

In [94]:
patents_merged = patents_merged[['patent_number', 'assignee', 'grant_year', 'application_year',
       'application_number', 'country', 'city', 'state', 'county_name', 'geoID',
       'ipc_section', 'team_size', 'inventors', 'men_inventors',
       'women_inventors']]

patents_merged

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county_name,geoID,ipc_section,team_size,inventors,men_inventors,women_inventors
0,7472557,CARRIER CORPORATION,2009,2004,11025351,US,FARMINGTON,CT,SAN JUAN,35045,F,4,3,3,0
1,7472744,CARRIER CORPORATION,2009,2005,11794273,US,FARMINGTON,CT,SAN JUAN,35045,F,2,1,1,0
2,7475757,OTIS ELEVATOR COMPANY,2009,2003,10554131,US,FARMINGTON,CT,SAN JUAN,35045,B,2,2,1,1
3,7479861,OTIS ELEVATOR COMPANY,2009,2003,10552383,US,FARMINGTON,CT,SAN JUAN,35045,H,10,10,8,2
4,7481069,CARRIER CORPORATION,2009,2005,11191679,US,FARMINGTON,CT,SAN JUAN,35045,F,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498686,D858729,"PELLET FIRE PIT, LLC",2019,2017,29592412,US,WEBSTER CITY,IA,HAMILTON,19079,NaN,1,1,1,0
1498687,D858794,"J & J PORTABLE SANITATION PRODUCTS, LLC",2019,2018,29639264,US,CRAWFORD,GA,OGLETHORPE,13221,NaN,4,4,3,1
1498688,D861112,2 LOTT'S L.L.C.,2019,2017,29598211,US,MINNEAPOLIS,KS,OTTAWA,20143,NaN,2,2,2,0
1498689,D868611,WALKABOUT DEVELOPMENTS LLC,2019,2018,29656834,US,FAULKTON,SD,FAULK,46049,NaN,1,1,1,0


In [96]:
#save it
#patents_merged.to_csv('patents.csv')

In [97]:
patents_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498691 entries, 0 to 1498690
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   patent_number       1498691 non-null  object
 1   assignee            1498691 non-null  object
 2   grant_year          1498691 non-null  int64 
 3   application_year    1498691 non-null  int64 
 4   application_number  1498691 non-null  int64 
 5   country             1498691 non-null  object
 6   city                1498691 non-null  object
 7   state               1498691 non-null  object
 8   county_name         1498691 non-null  object
 9   geoID               1498691 non-null  object
 10  ipc_section         1381395 non-null  object
 11  team_size           1498691 non-null  int64 
 12  inventors           1498691 non-null  int64 
 13  men_inventors       1498691 non-null  int64 
 14  women_inventors     1498691 non-null  int64 
dtypes: int64(7), object(8)
memory us